<a href="https://colab.research.google.com/github/LuisCGit/mit_bitcoinexpo_hackathon/blob/main/NFT_Neural_Network_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyyaml==5.4.1
! pip install sentence_transformers
! pip install img2vec_pytorch

In [36]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from img2vec_pytorch import Img2Vec
from PIL import Image
from scipy import spatial
import requests
import socket
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
socket.setdefaulttimeout(30) # 60 seconds

In [ ]:
#Function to generate embeddings for images using Resnet-18
def imagevec(i):
  print(i)
  if nftdf['url'][i][-3:]=='mov':
    url = 'https://ipfs.io/ipfs/'+nftdf['url'][0][7:]
  if nftdf['url'][i][7]!='i':
    url = 'https://ipfs.io/ipfs/'+nftdf['url'][i][7:]
  else:
    url = 'https://ipfs.io/ipfs/'+nftdf['url'][i][12:]
  try:
    urllib.request.urlretrieve(
      url,
      "gfg.png")
  except socket.timeout:
    return [0]*512
  except ConnectionResetError:
    imagevec(i)
    
  img = Image.open("gfg.png")
  im = Image.new('RGB', img.size, (255, 255, 255))
  im.paste(img, None)

  return img2vec.get_vec(im, tensor=False)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Generating embeddings for NFT Metadata 

In [4]:
#Loading a pre-trained BERT Model
model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
#load nft dataset
nftdf = pd.read_csv('drive/MyDrive/mitbitcoin_hackathon/nfts_100.csv').drop(columns=['Unnamed: 0'])
nftdf = nftdf.drop(columns = ['tokenId','recipient'])
nftdf = nftdf.dropna().reset_index().drop(columns=['index'])

## Embeddings for NFT name

In [6]:
name_embeddings = model.encode(nftdf['name'],normalize_embeddings = True, show_progress_bar=True,batch_size = 2*64)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

## Embeddings for NFT description

In [ ]:
#description embeddings
desc_embeddings = model.encode(nftdf['description'], normalize_embeddings=True, show_progress_bar=True, batch_size = 2*64)

## Embeddings for NFT attributes

In [ ]:
#attribute embeddings
att_embeddings = model.encode(nftdf['attributes'],normalize_embeddings = True, show_progress_bar=True, batch_size = 2*64)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

## Embeddings for NFT Image

In [ ]:
#image embeddings
vec = []
for i in range(0,nftdf.shape[0]):
  vec.append(imagevec(i))
  if i%5==0:
    pd.DataFrame(vec).to_csv('drive/MyDrive/mitbitcoin_hackathon/image_emb.csv',index=False)

In [19]:
image_emb = pd.read_csv('drive/MyDrive/mitbitcoin_hackathon/image_emb.csv')
image_emb = pd.concat([nftdf.iloc[:-1,:],image_emb],axis=1)

In [25]:
vec = []
for i in range(0,image_emb.shape[0]):
  if image_emb.iloc[i,5]!=0:
    vec.append(image_emb.iloc[i,:])

In [28]:
meta_imageembdf = pd.DataFrame(vec).reset_index().drop(columns=['index'])

In [30]:
imageembdf = meta_imageembdf.iloc[:,5:]

In [ ]:
nftdf2 = pd.concat([pd.DataFrame(name_embeddings),pd.DataFrame(desc_embeddings),pd.DataFrame(att_embeddings)],axis=1)

In [ ]:
nftdf2.to_csv('nftdf2.csv',index=False)

In [ ]:
nftdf2 = pd.read_csv('nftdf2.csv')

## Embeddings dimensionality reduction

In [ ]:
#pca = PCA(n_components = 3)
#nft_embed = pca.fit_transform(nftdf2)


#tsne 
nft_tsne = TSNE(n_components=3, n_jobs = -1).fit_transform(nftdf2)

In [ ]:
#for image
image_tsne = TSNE(n_components=3, n_jobs = -1).fit_transform(imageembdf)

In [38]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=6)

In [ ]:
#nft_labels = kmeans.fit_predict(nft_embed)
#nft_tsne_labels = kmeans.fit_predict(nft_tsne)
nft_tsne_labels = nftdf['contract']

In [39]:
#image
image_labels = kmeans.fit_predict(image_tsne)
imagetsnelabelsdf = pd.DataFrame(image_labels).rename(columns={0: "labels"})

In [ ]:
#labelsdf = pd.DataFrame(nft_labels).rename(columns={0: "labels"})
tsnelabelsdf = pd.DataFrame(nft_tsne_labels).rename(columns={0: "labels"})

In [ ]:
#pdf = pd.concat([pd.DataFrame(nft_embed),labelsdf],axis=1)
pdftsne = pd.concat([pd.DataFrame(nft_tsne),tsnelabelsdf],axis=1)

In [40]:
#pdfimagetsne
pdfimagetsne = pd.concat([pd.DataFrame(image_tsne),imagetsnelabelsdf],axis=1)

In [47]:
meta_imageembdf['contract']

0      SP1QK1AZ24R132C0D84EEQ8Y2JDHARDR58R72E1ZW.boom...
1      SP1QK1AZ24R132C0D84EEQ8Y2JDHARDR58R72E1ZW.boom...
2      SP1QK1AZ24R132C0D84EEQ8Y2JDHARDR58R72E1ZW.boom...
3      SP1QK1AZ24R132C0D84EEQ8Y2JDHARDR58R72E1ZW.boom...
4      SP1QK1AZ24R132C0D84EEQ8Y2JDHARDR58R72E1ZW.boom...
                             ...                        
710    SP3D0WY16NZXVTF8Q8849Y2R93GNWJ162RZF1D1VQ.poke...
711    SP3D0WY16NZXVTF8Q8849Y2R93GNWJ162RZF1D1VQ.poke...
712    SP3D0WY16NZXVTF8Q8849Y2R93GNWJ162RZF1D1VQ.poke...
713    SP3D0WY16NZXVTF8Q8849Y2R93GNWJ162RZF1D1VQ.poke...
714    SP3D0WY16NZXVTF8Q8849Y2R93GNWJ162RZF1D1VQ.poke...
Name: contract, Length: 715, dtype: object

In [ ]:
pdfimagetsne

In [51]:
#image embeddings
#Other text embeddings can be concatenated to image embeddings if needed
fig = px.scatter_3d(
    pdfimagetsne, x=0, y=1, z=2,
    color=meta_imageembdf['contract'], 
    labels={'influence_rating': 'Legend'},
    hover_data = [meta_imageembdf['name']]
)
fig.show()